# Frozen Lake

## Introduction

![FrozenLake](https://deeplizard.com/images/frozen%20lake%20winter.jpg)

In this Notebook, we will try to solve the famous Frozen Lake Problem. It is considered as a **Hello World** problem in Reinforcement Learning. We use Open AI Gym and Python to solve this Problem. Anyone with basic knowledge of Python and Reinforcement Learning can easily follow along.

![OpenAI](https://deeplizard.com/images/OpenAI%20logo.svg)

**Description of the Problem**

_Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend._ 

Let's have a look at our environment

In [ ]:
# Simple Python Code to demomstrate the Environment
from tabulate import tabulate
environment = [['S','F','F','F'],
               ['F','H','F','H'],
               ['F','F','F','H'],
               ['H','F','F','G']]
print(tabulate(environment,tablefmt="fancy_grid"))

╒═══╤═══╤═══╤═══╕
│ S │ F │ F │ F │
├───┼───┼───┼───┤
│ F │ H │ F │ H │
├───┼───┼───┼───┤
│ F │ F │ F │ H │
├───┼───┼───┼───┤
│ H │ F │ F │ G │
╘═══╧═══╧═══╧═══╛


S --> Starting position of the Agent (Safe, reward = 0)

F --> Frozen Part of Lake (Safe, reward = 0)

H --> Hole (Unsafe, terminates the episode, reward = 0)

G --> Goal (Safe, terminates the episode, reward = 1)

## Code

### Imports

In [43]:
# for setting up our environment
import gym   

# for building our policy
import random
import time

# for numerical computation
import numpy as np

# optional package, for better presentation
from IPython.display import clear_output

In [44]:
# initialize the environment object (Frozen Lake)
lake = gym.make("FrozenLake-v0")

In [45]:
# The Frozen Lake Environment by OPEN AI
# Red Spot denotes the position of the Agent
lake.render()


SFFF
FHFH
FFFH
HFFG


In [46]:
print(" The number of possible actions in the environment are {}".format(lake.action_space.n))

 The number of possible actions in the environment are 4


In [47]:
print(" The number of possible states in the environment are {}".format(lake.observation_space.n))

 The number of possible states in the environment are 16


### Create the Q Table

In [48]:
# Get the values for columns and rows for the Q Table
columns = lake.action_space.n
rows = lake.observation_space.n

In [49]:
# Initialize the Q Table with zero as the default value
q_table = np.zeros((rows, columns))

In [50]:
# Intializing the parameters
episodes = 10000
# Max possible steps in an episode
steps_per_episode = 100

# define the learning rate
alpha = 0.1

# define the discount rate
gamma = 0.99

# handling the exploration-exploitation problem
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

### Value Iteration Training

In [51]:
# list to store return of each episode
returns = list()

In [52]:
# Define the Policy for our agent
def policy():
  # threshold for exploration rate
  threshold = random.uniform(0, 1)

  if threshold > exploration_rate:
    # exploit    
    action = np.argmax(q_table[state,:])    
  else:
    # explore  
    action = lake.action_space.sample()
  
  return action
    

In [53]:
# outer "for loop" to control number of episodes
# and reset the environment after completing each episode
for episode in range(episodes):

  #resetting the environment for new episode
  state = lake.reset()

  # to keep track if the episode is finished or not
  flag = False

  # setting value of reward to zero for new episode
  current_reward = 0

  # The inner "for loop" is responsible to govern an individual episode
  for step in range(steps_per_episode):

    action = policy()

    # taking action
    new_state, reward, flag, info = lake.step(action)

    # update Q Table
    q_table[state, action] = (q_table[state, action] * (1 - alpha)) + \
    alpha * (reward + gamma * np.max(q_table[new_state, :]))

    # transition to the new state
    state = new_state

    # Adding reward for the current action to the reward for the episode
    current_reward += reward

    # check if the episode is completed, and start new episode if True
    if flag==True:
      break

  # we reduce the exploration rate as our agent 
  #learns more and more about the environment
  exploration_rate = min_exploration_rate + \
  (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

  # add the current episode reward to the return
  returns.append(current_reward)    

### Results

In [54]:
# Let's have a look at our final Q Table
print(q_table)

[[0.50436459 0.49067589 0.49070627 0.49024687]
 [0.38060542 0.41066292 0.35745717 0.47809896]
 [0.44020106 0.3951335  0.40622451 0.45688504]
 [0.28864601 0.37323245 0.15343251 0.44993588]
 [0.52693571 0.29136583 0.40151514 0.26030547]
 [0.         0.         0.         0.        ]
 [0.21250679 0.15162057 0.30840944 0.09645839]
 [0.         0.         0.         0.        ]
 [0.38745849 0.45413858 0.28094238 0.54779822]
 [0.36735147 0.56840381 0.49358777 0.36823383]
 [0.52936827 0.43043111 0.43100608 0.2847426 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.48221991 0.56164338 0.67940278 0.51543204]
 [0.70496801 0.86302654 0.77681041 0.72394217]
 [0.         0.         0.         0.        ]]


The problem with the Frozen Lake Environment is that the probability of the agent taking the action that it actually want to take is less. This is because the lake is "slippery"! To counter this, we make the following changes. These changes are purely based on the previous performance of the agent and these changes help the agent to perform better!

In [56]:
q_table[0,np.argmax(q_table[0])], q_table[0,1] = q_table[0,1], q_table[0,np.argmax(q_table[0])]

In [58]:
q_table[8,np.argmax(q_table[8])], q_table[8,0] = q_table[8,0], q_table[8,np.argmax(q_table[8])]

In [59]:
q_table

array([[0.49067589, 0.50436459, 0.49070627, 0.49024687],
       [0.38060542, 0.41066292, 0.35745717, 0.47809896],
       [0.44020106, 0.3951335 , 0.40622451, 0.45688504],
       [0.28864601, 0.37323245, 0.15343251, 0.44993588],
       [0.52693571, 0.29136583, 0.40151514, 0.26030547],
       [0.        , 0.        , 0.        , 0.        ],
       [0.21250679, 0.15162057, 0.30840944, 0.09645839],
       [0.        , 0.        , 0.        , 0.        ],
       [0.54779822, 0.45413858, 0.28094238, 0.38745849],
       [0.36735147, 0.56840381, 0.49358777, 0.36823383],
       [0.52936827, 0.43043111, 0.43100608, 0.2847426 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.48221991, 0.56164338, 0.67940278, 0.51543204],
       [0.70496801, 0.86302654, 0.77681041, 0.72394217],
       [0.        , 0.        , 0.        , 0.        ]])

In [60]:
# Let's calculate the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(returns),episodes/1000)
count = 1000

print("Average reward per thousand episodes\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

Average reward per thousand episodes

1000 :  0.05100000000000004
2000 :  0.20200000000000015
3000 :  0.4140000000000003
4000 :  0.5710000000000004
5000 :  0.6270000000000004
6000 :  0.6540000000000005
7000 :  0.6710000000000005
8000 :  0.7090000000000005
9000 :  0.6700000000000005
10000 :  0.6780000000000005


It is observed that the performance of the Agent improves with the increase in the number of episodes while the max exploration rate is capped at **0.8**. Also, the decay rate can be changed to 1e-3 to 1e-4 while the learning rate and discount rate are unchanged. These changes give the avg score of 0.75 per thousand episode and the max number of episodes required are greater than hundred thousand. Therefore, we stick to the previously defined parameters since the number of max episodes are only ten thousand.

## Visualization

In [64]:
# Outer loop resets everything for new episode
for episode in range(10):
    state = lake.reset()
    flag = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

  # Similar to our training loop, inner loop handles each individual episode
    for step in range(steps_per_episode):
      clear_output(wait=True)
      lake.render()
      time.sleep(0.3)

      # Taking action from our Q Table
      action = np.argmax(q_table[state,:])        
      new_state, reward, flag, info = lake.step(action)
      print("New State:{}".format(new_state))
      time.sleep(0.8)

      # Check if the episode is over
      if flag:
        clear_output(wait=True)
        lake.render()
        
        # reward is one, if the agent reaches the goal
        if reward == 1:
            print("Thanks Programmer! I reached the Goal!!")
            time.sleep(3)
        # reward is zero, if the agent fails
        else:
            print("Programmer, you failed to save me!")
            time.sleep(3)
            clear_output(wait=True)
        break

        # state transition
        state = new_state

# close the environment
lake.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
Programmer, you failed to save me!


## References

[Deep Lizard Tutorials](https://deeplizard.com/learn/playlist/PLZbbT5o_s2xoWNVdDudn51XM8lOuZ_Njv)